In [1]:
import pandas as pd
data=pd.read_excel('Bats_Data.xlsx', index_col=0) 

# Co-occurrence

In [2]:
def verify_cooccurence(i,j,dataframe):
    res=0
    for k in range(0,len(dataframe)-1):
        val1=list(dataframe.iloc[k])[7+i]
        val2=list(dataframe.iloc[k])[7+j]
        
        if (val1 != 0) and (val2 !=0):
            res=1
    
    return res

In [3]:
import pandas as pd
import networkx as nx
import math
import matplotlib.pyplot as plt

data=pd.read_excel('data1.xlsx', index_col=0)
years=[2011,2012,2013,2014]
#years=[2011]
#hab='1_ha fragment_interior'
#hab='10_ha fragment_interior'
#hab='100_ha fragment_interior'
hab='NA continuous_forest'
list_species=list(data.head(1))[7:48]


d_species_prop=dict()  # dict(species:{t1: p1, t2: p2 ......})
for i in list_species:
    d_species_prop[i]=dict()
    for j in years:
        d_species_prop[i][j]=0


dict_strength_connectivity2=dict()
for i in list_species:
    dict_strength_connectivity2[i]=dict()
    for j in years:
        dict_strength_connectivity2[i][j]=0
        
for year in years:
    if year==2011:
        d2=data.loc[(data['date'] > '2011-01-01') & (data['date'] < '2011-12-31') & (data['Hab'] == hab)]
    if year==2012:
        d2=data.loc[(data['date'] > '2012-01-01') & (data['date'] < '2012-12-31') & (data['Hab'] == hab)]
    if year==2013:
        d2=data.loc[(data['date'] > '2013-01-01') & (data['date'] < '2013-12-31') & (data['Hab'] == hab)]
    if year==2014:
        d2=data.loc[(data['date'] > '2014-01-01') & (data['date'] < '2014-12-31') & (data['Hab'] == hab)]
    
    nbr_samples=len(d2)-1


    #for sample_position in range(0,191):
    #    sample=list(data.iloc[sample_position])
    #    if (sample[6]==season && sample.year==year):
        
        
    list_edges=list()
        
    for i in range(0,len(list_species)):
        j=i+1
        while j<len(list_species):
            sp1=list_species[i]
            sp2=list_species[j]
        
            somme_sp1=sum(list(d2[sp1]))
            somme_sp2=sum(list(d2[sp2]))
            tot=somme_sp1+somme_sp2
        
            s=0
            for k in range(0,nbr_samples):
                s=s+abs(list(d2[sp1])[k]-list(d2[sp2])[k])
        
            if tot==0:
                weight=1
            else:
                weight=1-(s/tot)
            
            res=0
            res=verify_cooccurence(i,j,d2)
            
            if res == 1:
                list_edges.append((sp1,sp2,weight))
        
            j=j+1
        
     
    #print(list_edges)        
    g = nx.Graph()  
    g.add_weighted_edges_from(list_edges)
    #print(list(g.edges(data=True)))

    #if nx.is_empty(g)==False:
    #    dict_size[index_gsa].append(len(g))
    #    dict_edges[index_gsa].append(len(list(g.edges())))    
    #    dict_density[index_gsa].append(nx.density(g))
    #    dict_transitivity[index_gsa].append(nx.transitivity(g))
    #    dict_assortativity[index_gsa].append(nx.degree_assortativity_coefficient(g))
    #    dict_diameter[index_gsa].append(nx.diameter(g))
        



    dict_strength=dict() # node/specie: strength
    l_strg=list()
    for node in g:
        s=0
        l=g.edges(node, data=True)
        for e in l:
            s=s+e[2]['weight']
        dict_strength[node]=s
        l_strg.append(s)
         
    for node in g:
        dict_strength[node]=dict_strength[node]/max(l_strg)
    #print() 
    #print(dict_strength)
            
        
    dict_degree=nx.degree_centrality(g) # node/specie: degree
    #print()
    #print(dict_degree)
        
    dict_strength_connectivity=dict()   # node/specie: strength connectivity
        
    for i in dict_strength:
        dict_strength_connectivity[i]=math.sqrt(dict_strength[i]*dict_strength[i]+dict_degree[i]*dict_degree[i])
        dict_strength_connectivity2[i][year]=dict_strength_connectivity[i]
    #print()
    #print(dict_strength_connectivity)
    
    ptot=sum(list(dict_strength_connectivity.values()))
            
    for i in dict_strength_connectivity:
        if ptot==0:
            dict_strength_connectivity[i]=0
        else:
            dict_strength_connectivity[i]=dict_strength_connectivity[i]/ptot
        
        d_species_prop[i][year]=dict_strength_connectivity[i]
    #print()
    #print(dict_strength_connectivity)
    #print()
    #print(d_species_prop)  
    
    
    #width_o=15
    #hight_o=15
    #plt.subplots_adjust(left = 0.01, bottom = 0.01, right = 0.99, top = 0.99)
    #plt.figure(figsize=(width_o,hight_o))  # image is 8 x 8 inches
    #plt.axis('off')
    #nx.draw_networkx(g, with_labels=True, node_size=[v * 20000 for v in dict_strength_connectivity.values()] ,node_color=["#90EE90"], font_size=12, font_color='k', font_family='sans-serif', font_weight='bold')
    #plt.savefig('bats.jpg',dpi=300)
            
# computing temporal diversity            
s=0
for i in list_species:
    list_of_proportion=list(d_species_prop[i].values())
    set_of_proportion=set(list_of_proportion)
    #print(set_of_proportion)
    
    for j in set_of_proportion:
        if j!=0:
            s=s-((list_of_proportion.count(j)/4)*j*math.log(j))
    
s=math.exp(s)
    
#print(s)
print('Temporal Diversity: ',(s-1)/(40))
#print(d_species_prop)



# computing temporal variation of species    
#print(dict_strength_connectivity2)
for i in dict_strength_connectivity2:
    ptot=sum(list(dict_strength_connectivity2[i].values()))
    for j in dict_strength_connectivity2[i]:
        if ptot==0:
            dict_strength_connectivity2[i][j]=0
        else:
            dict_strength_connectivity2[i][j]=dict_strength_connectivity2[i][j]/ptot
#print(dict_strength_connectivity2)    


b=dict()  
for i in list_species:
    b[i]=0
    for j in dict_strength_connectivity2[i]:
        if dict_strength_connectivity2[i][j]!=0:
            b[i]=b[i]-dict_strength_connectivity2[i][j]*math.log(dict_strength_connectivity2[i][j])
    b[i]=math.exp(b[i])
    
btot=sum(list(b.values()))
print('Temporal variation of species: ',1-((btot-41)/(41*3)))

Temporal Diversity:  0.5639722958288795
Temporal variation of species:  0.46385395848851696


# Co-occurrence & functional distance

In [4]:
import pandas as pd
import networkx as nx

data_fct=pd.read_excel('gower.xlsx', index_col=0)
data=pd.read_excel('data1.xlsx', index_col=0)
years=[2011,2012,2013,2014]
#years=[2011]
hab='1_ha fragment_interior'
#hab='10_ha fragment_interior'
#hab='100_ha fragment_interior'
#hab='NA continuous_forest'
list_species=list(data.head(1))[7:48]


d_species_prop=dict()  # dict(species:{t1: p1, t2: p2 ......})
for i in list_species:
    d_species_prop[i]=dict()
    for j in years:
        d_species_prop[i][j]=0


dict_strength_connectivity2=dict()
for i in list_species:
    dict_strength_connectivity2[i]=dict()
    for j in years:
        dict_strength_connectivity2[i][j]=0
        
for year in years:
    if year==2011:
        d2=data.loc[(data['date'] > '2011-01-01') & (data['date'] < '2011-12-31') & (data['Hab'] == hab)]
    if year==2012:
        d2=data.loc[(data['date'] > '2012-01-01') & (data['date'] < '2012-12-31') & (data['Hab'] == hab)]
    if year==2013:
        d2=data.loc[(data['date'] > '2013-01-01') & (data['date'] < '2013-12-31') & (data['Hab'] == hab)]
    if year==2014:
        d2=data.loc[(data['date'] > '2014-01-01') & (data['date'] < '2014-12-31') & (data['Hab'] == hab)]
    
    nbr_samples=len(d2)-1


    #for sample_position in range(0,191):
    #    sample=list(data.iloc[sample_position])
    #    if (sample[6]==season && sample.year==year):
        
        
    list_edges=list()
        
    for i in range(0,len(list_species)):
        j=i+1
        while j<len(list_species):
            sp1=list_species[i]
            sp2=list_species[j]
        
            somme_sp1=sum(list(d2[sp1]))
            somme_sp2=sum(list(d2[sp2]))
            tot=somme_sp1+somme_sp2
        
            s=0
            for k in range(0,nbr_samples):
                s=s+abs(list(d2[sp1])[k]-list(d2[sp2])[k])
        
            if tot==0:
                weight=1
            else:
                weight=1-(s/tot)
                
            weight=weight*data_fct.loc[sp1].at[sp2]
            
            res=0
            res=verify_cooccurence(i,j,d2)
            
            if res == 1:
                list_edges.append((sp1,sp2,weight))
        
            j=j+1
        
     
    #print(list_edges)        
    g = nx.Graph()  
    g.add_weighted_edges_from(list_edges)
    #print(list(g.edges(data=True)))

    #if nx.is_empty(g)==False:
    #    dict_size[index_gsa].append(len(g))
    #    dict_edges[index_gsa].append(len(list(g.edges())))    
    #    dict_density[index_gsa].append(nx.density(g))
    #    dict_transitivity[index_gsa].append(nx.transitivity(g))
    #    dict_assortativity[index_gsa].append(nx.degree_assortativity_coefficient(g))
    #    dict_diameter[index_gsa].append(nx.diameter(g))
        



    dict_strength=dict() # node/specie: strength
    l_strg=list()
    for node in g:
        s=0
        l=g.edges(node, data=True)
        for e in l:
            s=s+e[2]['weight']
        dict_strength[node]=s
        l_strg.append(s)
         
    for node in g:
        dict_strength[node]=dict_strength[node]/max(l_strg)
    #print() 
    #print(dict_strength)
            
        
    dict_degree=nx.degree_centrality(g) # node/specie: degree
    #print()
    #print(dict_degree)
        
    dict_strength_connectivity=dict()   # node/specie: strength connectivity
        
    for i in dict_strength:
        dict_strength_connectivity[i]=math.sqrt(dict_strength[i]*dict_strength[i]+dict_degree[i]*dict_degree[i])
        dict_strength_connectivity2[i][year]=dict_strength_connectivity[i]
    #print()
    #print(dict_strength_connectivity)
    
    ptot=sum(list(dict_strength_connectivity.values()))
            
    for i in dict_strength_connectivity:
        if ptot==0:
            dict_strength_connectivity[i]=0
        else:
            dict_strength_connectivity[i]=dict_strength_connectivity[i]/ptot
        
        d_species_prop[i][year]=dict_strength_connectivity[i]
    #print()
    #print(dict_strength_connectivity)
    #print()
    #print(d_species_prop)  
            
# computing temporal diversity            
s=0
for i in list_species:
    list_of_proportion=list(d_species_prop[i].values())
    set_of_proportion=set(list_of_proportion)
    #print(set_of_proportion)
    
    for j in set_of_proportion:
        if j!=0:
            s=s-((list_of_proportion.count(j)/4)*j*math.log(j))
    
s=math.exp(s)
    
#print(s)
print('Temporal Diversity: ',(s-1)/(40))
#print(d_species_prop)



# computing temporal variation of species    
#print(dict_strength_connectivity2)
for i in dict_strength_connectivity2:
    ptot=sum(list(dict_strength_connectivity2[i].values()))
    for j in dict_strength_connectivity2[i]:
        if ptot==0:
            dict_strength_connectivity2[i][j]=0
        else:
            dict_strength_connectivity2[i][j]=dict_strength_connectivity2[i][j]/ptot
#print(dict_strength_connectivity2)    


b=dict()  
for i in list_species:
    b[i]=0
    for j in dict_strength_connectivity2[i]:
        if dict_strength_connectivity2[i][j]!=0:
            b[i]=b[i]-dict_strength_connectivity2[i][j]*math.log(dict_strength_connectivity2[i][j])
    b[i]=math.exp(b[i])
    
btot=sum(list(b.values()))
print('Temporal variation of species: ',1-((btot-41)/(41*3)))

Temporal Diversity:  0.20650412814186772
Temporal variation of species:  0.8504786336962136
